In [1]:
#solve memory issue

#do EDA to:
# - look for data cleaning opportunities
# - check the data

# additional datasets 
## apple mobility data
## case positive rate

In [2]:
import pandas as pd
import numpy as np

import requests
from io import StringIO 

from datetime import datetime

import os

from pathlib import Path
import shutil

pd.set_option('max_rows',100)

csv_path = '/kaggle/working/'

dict_df = {}

# Helper functions

In [3]:
def request_to_string(url):
    r = requests.get(url)
    StringData = StringIO(r.text)
    return StringData 

def show_country_name_differences(df):
    countries = df.index.get_level_values(0)
    missing_countries = countries[~countries.isin(df_country_join_key.iloc[:,0])].unique()
    missing_countries_str = ", ".join(missing_countries)
    print(f"Missing country names: {missing_countries_str}")

def add_column_to_join_key_mapping_df(mapping_dict,mapping_column):
    df_country_join_key[mapping_column] = df_country_join_key.index.map(mapping_dict)

    
def align_join_keys(align_key_column,df_tmp):
    rename_dict = df_country_join_key[df_country_join_key[align_key_column].notna()][['DataCommons_name_1',align_key_column]].set_index(align_key_column).to_dict()['DataCommons_name_1']
    
    df_tmp = df_tmp.rename(index=rename_dict,level=0)
    
    columns_to_drop = df_tmp.loc[~df_tmp.index.get_level_values(0).isin(df_country_join_key['DataCommons_name_1']),:].index
    df_tmp = df_tmp.drop(columns_to_drop,axis=0)
    return df_tmp

def make_path_and_save_csv(path, filename,df):
    Path(path).mkdir(parents=True, exist_ok=True)
    df.to_csv(f'{path}{filename}')

# OWID Panel

In [4]:
StringData = request_to_string('https://covid.ourworldindata.org/data/owid-covid-data.csv?raw=true')
dict_df['owid_panel'] = pd.read_csv(StringData,index_col=[2,3],parse_dates=[3])
make_path_and_save_csv(f'{csv_path}RAW/owid/','owid-covid-data-panel.csv',dict_df['owid_panel'])
dict_df['owid_panel'] = dict_df['owid_panel'].drop(columns=['iso_code','continent'])

# OWID Variants

In [5]:
StringData = request_to_string('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/variants/covid-variants.csv?raw=true')
df_owid_variants_raw = pd.read_csv(StringData,index_col=[0,1,2],parse_dates=[1])
make_path_and_save_csv(f'{csv_path}RAW/owid/','owid-covid-variants.csv',df_owid_variants_raw)


variants_of_concern = ['Alpha','Beta','Delta','Gamma']

dict_df['owid_variants'] = None

for variant in variants_of_concern:
    df_owid_variants_tmp = df_owid_variants_raw.loc[(slice(None),slice(None),variant),:]
    df_owid_variants_tmp = df_owid_variants_tmp.rename(columns={'num_sequences':f'{variant}_num_sequences','perc_sequences':f'{variant}_perc_sequences'})
    df_owid_variants_tmp = df_owid_variants_tmp.droplevel('variant')

    if dict_df['owid_variants'] is None:
        dict_df['owid_variants'] = df_owid_variants_tmp[['num_sequences_total',f'{variant}_num_sequences',f'{variant}_perc_sequences']]
    else:
        dict_df['owid_variants'] = dict_df['owid_variants'].merge(df_owid_variants_tmp.drop(['num_sequences_total'],axis=1),left_index=True,right_index=True)

df_owid_variants_raw = None #save memory
df_owid_variants_tmp = None #save memory

# OWID Vaccinations by Manufacturer

In [6]:
StringData = request_to_string('https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations-by-manufacturer.csv?raw=true')
df_owid_vaccines_raw = pd.read_csv(StringData,index_col=[0,1,2],parse_dates=[1])
make_path_and_save_csv(f'{csv_path}RAW/owid/','owid-covid-vaccines.csv',df_owid_vaccines_raw)

vaccines = df_owid_vaccines_raw.index.get_level_values(2).unique()


dict_df['owid_vaccines'] = None

for vaccine in vaccines:
    df_owid_vaccines_tmp = df_owid_vaccines_raw.loc[(slice(None),slice(None),vaccine),:]
    df_owid_vaccines_tmp = df_owid_vaccines_tmp.rename(columns={'total_vaccinations':f'{vaccine}_total_vaccinations'})
    df_owid_vaccines_tmp[f'{vaccine}_new_vaccinations'] = df_owid_vaccines_tmp[f'{vaccine}_total_vaccinations'].diff(1)
    df_owid_vaccines_tmp = df_owid_vaccines_tmp.droplevel('vaccine')

    if dict_df['owid_vaccines'] is None:
        dict_df['owid_vaccines'] = df_owid_vaccines_tmp
    else:
        dict_df['owid_vaccines'] = dict_df['owid_vaccines'].merge(df_owid_vaccines_tmp,left_index=True,right_index=True,how='outer')

df_owid_vaccines_tmp = None
df_owid_vaccines_raw = None

# Google Mobility Data

## Download and parse data
This is a large file

In [7]:
StringData = request_to_string('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv')
#df_goog_mobility = pd.read_csv(StringData,usecols=[1,2,8,9,10,11,12,13,14],parse_dates=[8])
df_goog_mobility = pd.read_csv(StringData,parse_dates=[8])
StringData = None
make_path_and_save_csv(f'{csv_path}RAW/google/','google-mobility.csv',df_goog_mobility)
df_goog_mobility = df_goog_mobility[df_goog_mobility['sub_region_1'].isna()] #remove subregions

mobility_columns = ['retail_and_recreation_percent_change_from_baseline','grocery_and_pharmacy_percent_change_from_baseline','parks_percent_change_from_baseline','transit_stations_percent_change_from_baseline','workplaces_percent_change_from_baseline','residential_percent_change_from_baseline']
df_goog_mobility = df_goog_mobility.groupby(['country_region', 'date'])[mobility_columns].mean()


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
rename_dict = {"CÃ´te d'Ivoire": "Cote d'Ivoire", "Myanmar (Burma)" : "Myanmar" ,'The Bahamas' : "Bahamas"}
df_goog_mobility = df_goog_mobility.rename(index=rename_dict,level=0)

countries_to_drop = ['Puerto Rico','RÃ©union']

columns_to_drop = df_goog_mobility.loc[df_goog_mobility.index.get_level_values(0).isin(countries_to_drop),:].index
df_goog_mobility = df_goog_mobility.drop(columns_to_drop,axis=0)
df_goog_mobility.index.rename(['location','date'],inplace=True)

dict_df['goog_mobility'] = df_goog_mobility
df_goog_mobility = None

# Combine into a single DataFrame

In [9]:
def get_global_list(dict_df,level):
    global_list = np.empty(0,dtype=np.datetime64)
    
    for df in dict_df:
        global_list = np.concatenate((global_list,np.array(dict_df[df].index.get_level_values(level).unique())))        
        
    return sorted(np.unique(global_list))


In [10]:
countries= get_global_list(dict_df,0)
dates = get_global_list(dict_df,1)
date_range = pd.date_range(min(dates),max(dates))
idx = pd.MultiIndex.from_product([countries,date_range],names=['location','date'])


In [11]:

df_covid = pd.DataFrame(index=idx)
for df in dict_df:
    df_covid = df_covid.merge(dict_df[df],left_index=True,right_index=True,how='outer')

dict_df = None


## Add calculated field for vaccines

In [12]:
for vaccine in vaccines:
    df_covid[f'{vaccine}_perc_of_total_vaccinations'] = df_covid[f'{vaccine}_total_vaccinations']/df_covid['total_vaccinations']
    df_covid[f'{vaccine}_perc_of_new_vaccinations'] = df_covid[f'{vaccine}_new_vaccinations']/df_covid['new_vaccinations']

## Create monolithic DataFrame and save to CSV

In [13]:
columns_ordered = sorted(df_covid.columns)
df_covid = df_covid.reindex(columns=columns_ordered)


In [14]:
df_covid.to_csv('COVID_panel.csv')